In [7]:
import tensorflow as tf
import data_util
import cv2
import matplotlib.pyplot as plt
import resnet_V2
from tensorflow.keras.utils import plot_model
from tensorflow import keras

## 데이터 불러오기

In [32]:
train_ds = data_util.get_cu_dataset(train_type='landmark')

## 모델 학습

In [21]:
num_class = 16

In [43]:
en = resnet_V2.ResNetEncoder()

In [45]:
resnet = keras.models.Sequential()
resnet.add(en)
resnet.add(keras.layers.Flatten())
# resnet.add(keras.layers.Dense(256))
resnet.add(keras.layers.Dense(num_class))
resnet.compile(loss=keras.losses.mse,
              optimizer='adam',
              metrics=['accuracy'])
# temp = tf.random.normal(shape=(1, 256, 256, 3))
# resnet(temp)
# resnet.summary()

In [59]:
# for i in train_ds.take(1):
    # for j in 
resnet.fit(train_ds, epochs=1)

KeyboardInterrupt: 